In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
import os

# TODO: Fill in the Google Drive path where you uploaded
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'ML'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'MyDrive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))

Mounted at /content/drive/
['settings.py', 'Youtube01-Psy.csv', 'Youtube03-LMFAO.csv', 'Youtube02-KatyPerry.csv', 'Youtube04-Eminem.csv', 'Youtube05-Shakira.csv', 'get_comment_data.py', 'vectorize_data.py', 'vec_data_no_norm.py', 'initial_eda.py', 'word2vecdat.csv', 'word2vecdat_nopunc.csv', 'data_with_vectors.csv', 'data_with_vectors2.csv', 'data.csv', 'word2vec_model', 'word2vecdat_googlemodel.csv', 'data2.csv', 'ML_word2vec_model.ipynb', 'ted_hadley_Assignment_8.ipynb', 'ML_boosted_model_and_neural_net.ipynb', 'word2vec_vectorize']


In [ ]:
import pandas as pd
import numpy as np
import datetime
import copy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize

In [ ]:
# Get Data in workable form

# Day videos were posted
day_p = datetime.datetime.strptime('07-15-2012', '%m-%d-%Y') 
day_k = datetime.datetime.strptime('09-05-2013', '%m-%d-%Y') 
day_l = datetime.datetime.strptime('03-09-2011', '%m-%d-%Y') 
day_e = datetime.datetime.strptime('08-05-2010', '%m-%d-%Y') 
day_s = datetime.datetime.strptime('06-04-2010', '%m-%d-%Y')
dates = [day_p,day_k,day_l,day_e,day_s]

#func

lst = ['1-Psy','2-KatyPerry','3-LMFAO','4-Eminem','5-Shakira']
data_files = [GOOGLE_DRIVE_PATH + '/Youtube0' + l + '.csv' for l in lst]
dat = pd.read_csv(data_files[0])
dat['VIDEO_AUTHOR'] = ['Psy']*dat.shape[0]
time_elapsed = []
time_of_day = []
for t in dat['DATE']:
    if type(t) != str:
        time_elapsed.append(0)
        time_of_day.append(0)
    else:
        date = datetime.datetime.strptime(t, '%Y-%m-%dT%H:%M:%S')
        time_elapsed.append((date - dates[0]).total_seconds())
        time_of_day.append(date.hour*60*60+date.minute*60+date.second)
dat['TIME'] = time_of_day
dat['TIME_ELAPSED'] = time_elapsed
for i in range(1,5):
    df = pd.read_csv(data_files[i])
    df['VIDEO_AUTHOR'] = [lst[i][2:]]*df.shape[0]
    time_elapsed = []
    time_of_day = []
    for t in df['DATE']:
        if type(t) != str:
            time_elapsed.append(0)
            time_of_day.append(0)
        else:
            if len(t) == 19:
                date = datetime.datetime.strptime(t, '%Y-%m-%dT%H:%M:%S')
            else:
                date = datetime.datetime.strptime(t, '%Y-%m-%dT%H:%M:%S.%f')
            time_elapsed.append((date - dates[i]).total_seconds())
            time_of_day.append(date.hour*60*60+date.minute*60+date.second)
    # Deal with missing data        
    num_fails = sum([t == 0 for t in time_of_day])
    l = len(time_of_day)
    sum_t = sum(time_of_day)
    sum_t_e = sum(time_elapsed)
    avg_t = sum_t / (l - num_fails)
    avg_t_e = sum_t_e / (l - num_fails)
    def rep_missing_t(x):
        if x == 0:
            return avg_t
        return x
    def rep_missing_t_e(x):
        if x == 0:
            return avg_t_e
        return x
    time_of_day = list(map(rep_missing_t, time_of_day))
    time_elapsed = list(map(rep_missing_t_e, time_elapsed))
    df['TIME'] = time_of_day
    df['TIME_ELAPSED'] = time_elapsed
    dat = pd.concat([dat,df], ignore_index = True)

In [ ]:
import gensim
from gensim.models import Word2Vec, KeyedVectors
from string import punctuation, whitespace
import re
import itertools
from nltk.tokenize.casual import TweetTokenizer

In [ ]:
# Create Word2Vec Model

import gensim.downloader as api

#w2v = api.load('word2vec-google-news-300')

txt_data = pd.DataFrame(dat['CONTENT'])
txt_data.columns = ['response']
#print(txt_data.response[0:10])

t = TweetTokenizer(preserve_case = False, reduce_len = True, )

txt_pp = txt_data.response.apply(t.tokenize)
#print(txt_pp)

# Build Word2Vec model for t ext vectorization
model=gensim.models.Word2Vec(size = 500, window = 5, min_count = 1, workers= 4, sg = 0)

# Train vectorization model
model.build_vocab(txt_pp, progress_per = 1000)
model.train(txt_pp, total_examples = model.corpus_count, epochs = model.epochs)

(135563, 192650)

In [ ]:
txt_data.response[0]

'Huh, anyway check out this you[tube] channel: kobyoshi02'

In [ ]:
txt_pp

0       [huh, ,, anyway, check, out, this, you, [, tub...
1       [hey, guys, check, out, my, new, channel, and,...
2         [just, for, test, i, have, to, say, murdev.com]
3       [me, shaking, my, sexy, ass, on, my, channel, ...
4       [watch, ?, v, =, vtarggvgtwq, check, this, out...
                              ...                        
1951    [i, love, this, song, because, we, sing, it, a...
1952    [i, love, this, song, for, two, reasons, :, 1....
1953                                                [wow]
1954                        [shakira, u, are, so, wiredo]
1955                     [shakira, is, the, best, dancer]
Name: response, Length: 1956, dtype: object

In [ ]:
model.wv[':P']

array([ 0.01994668, -0.02469195, -0.03637112, -0.00285167,  0.06823139,
        0.06467104,  0.02609558,  0.04581076,  0.04358261, -0.00404733,
        0.0528858 ,  0.0038912 , -0.03691358,  0.0073606 ,  0.0169463 ,
        0.01080052,  0.02408598,  0.01256848,  0.01813402,  0.02724983,
       -0.01029994,  0.05127378, -0.01514513, -0.01032874,  0.02178437,
        0.01640887, -0.03331416,  0.07574742, -0.00210561, -0.01266107,
       -0.03128713, -0.05298591,  0.02899907,  0.04224429,  0.00220195,
        0.03555434,  0.02188013,  0.00475518, -0.04970045,  0.03294488,
        0.00649875, -0.01685259, -0.00960779, -0.04725459,  0.0419822 ,
       -0.02324245,  0.00544016, -0.02361427, -0.02422095,  0.00691766,
        0.00405259,  0.02482101, -0.00910235,  0.02441922, -0.02689964,
        0.03334179, -0.02982478,  0.02948835, -0.00257538,  0.02823376,
        0.04965916,  0.03269792,  0.00880523, -0.04465226, -0.03595839,
        0.01761963, -0.02095437,  0.01111722, -0.04825033,  0.03

In [ ]:
model.most_similar('favorite', topn=5)

<ipython-input-34-6c9b2364f26d>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  model.most_similar('favorite', topn=5)


[('channel', 0.9940526485443115),
 ('miss', 0.9939637184143066),
 ('post', 0.9939528703689575),
 ('funny', 0.9939417839050293),
 ('plz', 0.993928074836731)]

In [ ]:
# Vectorize data with Word2Vec model
vec_dat = []
for i in range(txt_pp.shape[0]):
  vec_array = []
  not_in_vocab = 0
  for word in txt_pp[i]:
    try:
      vec = model.wv[word]
      vec_array.append(vec)
    except:
      print(i)
      not_in_vocab += 1
      continue
  row = np.array(vec_array).mean(axis = 0)
  
  vec_dat.append(copy.deepcopy(row))
  if not_in_vocab >= 1:
    print(i)




In [ ]:
# To dataframe
vec_dat = pd.DataFrame(vec_dat)

In [ ]:
vec_dat['CLASS'] = dat['CLASS']

In [ ]:
# Combine with Target variable
vec_dat


,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,CLASS
0,0.114079,-0.228967,-0.279277,0.082889,0.444362,0.443761,0.444164,0.430075,0.271988,-0.227148,...,-0.012437,0.058258,-0.075945,-0.462967,0.161112,-0.064415,0.187151,-0.310874,-0.255008,1
1,0.180013,-0.357527,-0.456233,0.109179,0.703562,0.718084,0.734013,0.663556,0.421321,-0.345483,...,-0.016669,0.068326,-0.122440,-0.766037,0.251632,-0.104447,0.297376,-0.490502,-0.424372,1
2,0.163188,-0.315209,-0.405540,0.085783,0.625167,0.649495,0.665627,0.574989,0.368471,-0.300681,...,-0.013007,0.045420,-0.106966,-0.694516,0.217979,-0.096593,0.259570,-0.429932,-0.388006,1
3,0.118237,-0.239201,-0.294853,0.076970,0.464623,0.472914,0.476054,0.444898,0.283861,-0.234756,...,-0.014409,0.055466,-0.078484,-0.491982,0.164310,-0.065165,0.196359,-0.324925,-0.269853,1
4,0.184927,-0.357216,-0.442399,0.105057,0.709985,0.709138,0.719823,0.668578,0.418534,-0.352473,...,-0.019742,0.061381,-0.126769,-0.762444,0.242392,-0.089494,0.313624,-0.498419,-0.398085,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1951,0.174269,-0.331616,-0.431051,0.095258,0.664189,0.679617,0.699583,0.613643,0.392669,-0.319554,...,-0.011602,0.044154,-0.118422,-0.736685,0.227976,-0.099639,0.285044,-0.459257,-0.404983,0
1952,0.143214,-0.272336,-0.346110,0.077753,0.545114,0.554748,0.566238,0.502385,0.318591,-0.262701,...,-0.012276,0.037046,-0.095560,-0.599663,0.184289,-0.078847,0.232593,-0.373565,-0.325109,0
1953,0.038279,-0.075736,-0.095621,0.025276,0.156835,0.152892,0.157790,0.145502,0.093869,-0.073470,...,-0.003516,0.008560,-0.029254,-0.175128,0.052275,-0.018876,0.066925,-0.109847,-0.087766,0
1954,0.130916,-0.252160,-0.327409,0.071833,0.501279,0.517884,0.530867,0.465231,0.295220,-0.240187,...,-0.013905,0.035298,-0.089489,-0.555447,0.173003,-0.078529,0.213307,-0.343053,-0.304045,0


In [ ]:
# Save vectorized data as CSV file
vec_dat.to_csv(GOOGLE_DRIVE_PATH +'/word2vecdat.csv')

In [ ]:
# Repeat but excluding punctuation/emojis

txt_data = pd.DataFrame(dat['CONTENT'])
txt_data.columns = ['response']
#print(txt_data.response[0:10])

txt_pp = txt_data.response.apply(gensim.utils.simple_preprocess)
#print(txt_pp)

# Build Word2Vec model for text vectorization
model=gensim.models.Word2Vec(window = 5, min_count = 1, workers= 4, sg = 0)

# Train vectorization model
model.build_vocab(txt_pp, progress_per = 1000)
model.train(txt_pp, total_examples = model.corpus_count, epochs = model.epochs)

(112893, 150105)

In [ ]:
txt_pp.index = range(1956)
txt_pp.index

RangeIndex(start=0, stop=1956, step=1)

In [ ]:
# Remove data instances of just emojis or emoticons
to_drop = [134,585,716,825,844,900,1220,1265,1386,1538,1549,1821,1825]
txt_pp = txt_pp.drop(to_drop).reset_index()

In [ ]:
txt_pp

,index,response
0,0,"[huh, anyway, check, out, this, you, tube, cha..."
1,1,"[hey, guys, check, out, my, new, channel, and,..."
2,2,"[just, for, test, have, to, say, murdev, com]"
3,3,"[me, shaking, my, sexy, ass, on, my, channel, ..."
4,4,"[watch, vtarggvgtwq, check, this, out]"
...,...,...
1938,1951,"[love, this, song, because, we, sing, it, at, ..."
1939,1952,"[love, this, song, for, two, reasons, it, is, ..."
1940,1953,[wow]
1941,1954,"[shakira, are, so, wiredo]"


In [ ]:
vec_dat = []
for comment in txt_pp.response:
  #print(comment)
  vec_array = []
  not_in_vocab = 0
  for word in comment:
    try:
      vec = model.wv[word]
      vec_array.append(vec)
    except:
      print(i)
      not_in_vocab += 1
      continue
  row = np.array(vec_array).mean(axis = 0)
  
  vec_dat.append(copy.deepcopy(row))
  if len(comment) == 0:
    print(i)

In [ ]:
vec_dat = pd.DataFrame(vec_dat)
vec_dat['CLASS'] = dat['CLASS']

In [ ]:
# Save vectorized data as CSV file
vec_dat.to_csv(GOOGLE_DRIVE_PATH +'/word2vecdat_nopunc.csv')

In [ ]:
# Using pre-trained Google Model & Tweet tokenizer

txt_data = pd.DataFrame(dat['CONTENT'])
txt_data.columns = ['response']
#print(txt_data.response[0:10])



In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('stopwords')
nltk.download('punkt')
def remove_stop_words(sentence):
  stop_words = set(stopwords.words('english'))
  word_tokens = word_tokenize(sentence)
  filt = [w for w in word_tokens if not w.lower() in stop_words]
  new_sentence = ' '.join(filt)
  return new_sentence

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
for i in range(txt_data.shape[0]):
  txt_data.response[i] = remove_stop_words(txt_data.response[i])

In [ ]:
txt_data

,response
0,"Huh , anyway check [ tube ] channel : kobyoshi02"
1,Hey guys check new channel first vid US MONKEY...
2,test say murdev.com
3,shaking sexy ass channel enjoy ^_^ ﻿
4,watch ? v=vtaRGgvGtWQ Check .﻿
...,...
1951,love song sing Camp time ! !
1952,love song two reasons : 1.it Africa 2.i born b...
1953,wow
1954,Shakira u wiredo


In [ ]:
t = TweetTokenizer(preserve_case = False, reduce_len = True, )

txt_pp = txt_data.response.apply(t.tokenize)
#print(txt_pp)

# Build Word2Vec model for text vectorization

In [ ]:
txt_pp.shape

(1956,)

In [ ]:
vec_dat = []
for comment in txt_pp:
  #print(comment)
  vec_array = []
  not_in_vocab = 0
  for word in comment:
    try:
      vec = model.wv[word]
      vec_array.append(vec)
    except:
      print(word)
      not_in_vocab += 1
      continue
  row = np.array(vec_array).mean(axis = 0)
  
  vec_dat.append(copy.deepcopy(row))
  if len(comment) == 0:
    print('ohno')

https
twitter.com/gbphotographygb﻿
https
premium.easypromosapp.com/voteme/19924/616375350﻿
http
www.ebay.com/itm/171183229277
sspagename
strk
meselx
amp
_trksid
=p
3984
m1555
l2649
http
ubuntuone.com/40beuutvu2zkxk4utgpz8k﻿
http
www.teespring.com/tigermeathoodie﻿
http
audiojungle.net/user/eugenekalinin/portfolio
ref
eugenekalinin
ll
ll
http
www.twitch.tv/jaroadc
... ... ... ...
http
www.avaaz.org/po/petition/youtube_corporation_fox_broadcasting_company_anular_os_strikes_no_canal_nostalgia/
cxpzpgb
https
www.facebook.com/pages/brew-crew-2014/134470083389909
http
www.change.org/petitions/the-national-football-league-allow-gwar-to-perform-the-2015-super-bowl-halftime-show
http
www.twitch.tv/sevadus﻿
http
tankionline.com
cd92db3f4
gon
na
n't
http
www.ebay.com/itm/131338190916
sspagename
strk
meselx
amp
_trksid
=p
3984
m1555
l2649
http
www.guardalo.org/best-of-funny-cats-gatti-pazzi-e-divertenti-2013-5287/1000415527985/
http
hackfbaccountlive.com/
ref
4604617
amp
gt
gt
gt
gt
https
www.faceb

In [ ]:
vec_dat = pd.DataFrame(vec_dat)
vec_dat['CLASS'] = dat['CLASS']
vec_dat.to_csv(GOOGLE_DRIVE_PATH +'/word2vecdat_googlemodel.csv')

In [ ]:
vec_dat

,0,1,2,3,4,5,6,7,8,9,...,491,492,493,494,495,496,497,498,499,CLASS
0,0.077913,-0.022293,0.057459,-0.010824,-0.152638,0.016702,0.028469,0.041638,-0.049569,-0.029329,...,0.065318,-0.135188,0.052165,0.085576,-0.009956,-0.170830,-0.032572,0.078751,0.003538,1
1,0.137251,-0.036530,0.099021,-0.017143,-0.258145,0.034006,0.047922,0.072656,-0.087520,-0.051619,...,0.111034,-0.233096,0.087714,0.146309,-0.020354,-0.291288,-0.055167,0.136498,0.011399,1
2,0.013548,-0.003312,0.009713,-0.001287,-0.025333,0.004266,0.004537,0.007070,-0.008147,-0.004741,...,0.010972,-0.022506,0.008094,0.014582,-0.002488,-0.028498,-0.005632,0.013757,0.001557,1
3,0.060982,-0.017799,0.043934,-0.010218,-0.109201,0.016486,0.021335,0.031995,-0.039464,-0.019954,...,0.047317,-0.101474,0.037543,0.065143,-0.010541,-0.128366,-0.023979,0.060039,0.005375,1
4,0.170526,-0.054545,0.126256,-0.034645,-0.296776,0.045117,0.060573,0.089763,-0.109833,-0.054029,...,0.131273,-0.282139,0.103078,0.181938,-0.031918,-0.356947,-0.066504,0.168050,0.012392,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1951,0.124069,-0.034620,0.087077,-0.019107,-0.219600,0.034941,0.042588,0.064691,-0.079691,-0.040889,...,0.094649,-0.204157,0.074706,0.130702,-0.022051,-0.255329,-0.047426,0.120760,0.012423,0
1952,0.096124,-0.028041,0.068619,-0.017652,-0.169311,0.027401,0.033804,0.050056,-0.062022,-0.029836,...,0.073328,-0.158632,0.057450,0.102839,-0.017133,-0.199680,-0.036425,0.094510,0.008356,0
1953,0.046170,-0.012527,0.032121,-0.007257,-0.080549,0.011983,0.016419,0.023473,-0.028599,-0.013752,...,0.034579,-0.076251,0.026821,0.048221,-0.008637,-0.094915,-0.018249,0.043942,0.003304,0
1954,0.072673,-0.019580,0.051737,-0.011545,-0.131662,0.019090,0.026097,0.038054,-0.046453,-0.025426,...,0.057589,-0.121878,0.044139,0.077410,-0.012186,-0.151184,-0.028857,0.072356,0.007194,0
